In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('motorcycle_sales.csv')

In [3]:
df.head()

,order_number,date,warehouse,client_type,product_line,quantity,unit_price,total,payment,payment_fee
0,N1,2021-06-01T00:00:00.000Z,North,Retail,Breaking system,9,19.29,173.61,Cash,0.00
1,N2,2021-06-01T00:00:00.000Z,North,Retail,Suspension & traction,8,32.93,263.45,Credit card,0.03
2,N3,2021-06-01T00:00:00.000Z,North,Wholesale,Frame & body,16,37.84,605.44,Transfer,0.01
3,N4,2021-06-01T00:00:00.000Z,North,Wholesale,Suspension & traction,40,37.37,1494.80,Transfer,0.01
4,N5,2021-06-01T00:00:00.000Z,North,Retail,Frame & body,6,45.44,272.61,Credit card,0.03


In [4]:
from sqlalchemy import create_engine, text


user = 'postgres'
password = 'admin'
host = 'localhost'
port = '5432'

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/postgres')
conn = engine.connect()

conn.execution_options(isolation_level="AUTOCOMMIT").execute(text("CREATE DATABASE motorcycle_sales;"))

conn.close()

In [5]:
db_engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/motorcycle_sales')
df.to_sql('sales',db_engine, if_exists='replace',index=False)

1000

In [ ]:
query = """
SELECT COUNT(*)
FROM sales
"""

with db_engine.connect() as conn:
    run = conn.execute(text(query))
    result = pd.DataFrame(run.fetchall(), columns=run.keys())

result

,count
0,1000


In [10]:
query = """ 
SELECT 
    product_line, 
    TO_CHAR(date::timestamp,'Month'), 
    warehouse,
    SUM(total) - SUM(payment_fee) AS net_revenue
FROM sales
WHERE client_type = 'Wholesale'
GROUP BY product_line, TO_CHAR(date::timestamp,'Month'), warehouse
ORDER BY product_line
"""

with db_engine.connect() as conn:
    run = conn.execute(text(query))
    result = pd.DataFrame(run.fetchall(), columns=run.keys())

result

,product_line,to_char,warehouse,net_revenue
0,Breaking system,July,North,2594.44
1,Breaking system,August,West,2500.67
2,Breaking system,July,West,3060.93
3,Breaking system,July,Central,3778.65
4,Breaking system,August,North,1770.84
5,Breaking system,June,West,1212.75
6,Breaking system,June,North,1487.77
7,Breaking system,June,Central,3684.89
8,Breaking system,August,Central,3039.41
9,Electrical system,July,West,449.46
